In [2]:
import pandas as pd
import numpy as np
import random
import uuid
import re

# Load base MIMIC-III CSVs
diagnoses = pd.read_csv('data/DIAGNOSES_ICD.csv',usecols=['SUBJECT_ID','HADM_ID','ICD9_CODE'])  # columns: SUBJECT_ID, HADM_ID, ICD9_CODE
admissions = pd.read_csv('data/ADMISSIONS.csv',usecols=['SUBJECT_ID','HADM_ID','DIAGNOSIS'])    # columns: SUBJECT_ID, HADM_ID
icd_desc = pd.read_csv('data/D_ICD_DIAGNOSES.csv',usecols=['ICD9_CODE','SHORT_TITLE']) # columns: ICD9_CODE, SHORT_TITLE, LONG_TITLE

# Only load needed columns from PRESCRIPTIONS to save memory
prescriptions = pd.read_csv(
    'data/PRESCRIPTIONS.csv',
    usecols=['SUBJECT_ID', 'HADM_ID', 'DRUG', 'NDC', 'PROD_STRENGTH']
)
patients = pd.read_csv('data/PATIENTS.csv',usecols=['SUBJECT_ID','GENDER'])  # columns: SUBJECT_ID, GENDER

# Merge ICD descriptions into diagnoses
diagnoses = diagnoses.merge(icd_desc, on='ICD9_CODE', how='inner')

# Merge diagnoses with admissions
df = diagnoses.merge(admissions, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

# Merge prescriptions using SUBJECT_ID and HADM_ID to avoid explosion
df = df.merge(prescriptions, on=['SUBJECT_ID', 'HADM_ID'], how='inner', suffixes=('_diag', '_rx'))

# Merge with patient demographics
df = df.merge(patients[['SUBJECT_ID', 'GENDER']], on='SUBJECT_ID', how='inner')

# Filter out rows with missing drug names
df = df.dropna(subset=['DRUG'])

# Generate patient-level features
unique_subjects = df[['SUBJECT_ID', 'GENDER']].drop_duplicates().copy()
unique_subjects['PatientID'] = [str(uuid.uuid4()) for _ in range(len(unique_subjects))]
unique_subjects['Age'] = np.random.randint(18, 90, size=len(unique_subjects))
unique_subjects['Weight_kg'] = np.round(np.random.normal(70, 15, len(unique_subjects)), 1)
unique_subjects['Height_cm'] = np.round(np.random.normal(170, 10, len(unique_subjects)), 1)
unique_subjects['BMI'] = np.round(
    unique_subjects['Weight_kg'] / ((unique_subjects['Height_cm'] / 100) ** 2), 1
)
chronic_map = (
    diagnoses[['SUBJECT_ID', 'ICD9_CODE']]
    .merge(icd_desc[['ICD9_CODE', 'SHORT_TITLE']], on='ICD9_CODE', how='left')
    .dropna(subset=['SHORT_TITLE'])
    .drop_duplicates()
    .groupby('SUBJECT_ID')['SHORT_TITLE']
    .apply(lambda x: ', '.join(x.unique()[:3]))  # Limit to top 3 unique diagnoses
    .reset_index(name='Chronic_Conditions')
)

# # Step 2: Merge into the unique_subjects DataFrame (one row per patient)
unique_subjects = unique_subjects.merge(chronic_map, on='SUBJECT_ID', how='left')

def generate_drug_allergy(pool):
    """If 'None' is selected, return only 'None'. Otherwise, return a combo without 'None'."""
    if random.random() < 0.4:  # Adjust probability of having no allergies (e.g., 40%)
        return 'None'
    else:
        non_none_allergies = [a for a in pool if a != 'None']
        k = random.randint(1, min(2, len(non_none_allergies)))
        return ', '.join(random.sample(non_none_allergies, k)) 
# # Fill missing with 'None' if any
unique_subjects['Chronic_Conditions'] = unique_subjects['Chronic_Conditions'].fillna('None')
allergy_pool = ['Penicillin', 'Aspirin','Shellfish', 'None']
unique_subjects['Drug_Allergies'] = [generate_drug_allergy(allergy_pool) for _ in range(len(unique_subjects))]
# Record number of rows for later use
num_rows = df.shape[0]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8953467 entries, 0 to 8953466
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   SUBJECT_ID     int64  
 1   HADM_ID        int64  
 2   ICD9_CODE      object 
 3   SHORT_TITLE    object 
 4   DIAGNOSIS      object 
 5   DRUG           object 
 6   NDC            float64
 7   PROD_STRENGTH  object 
 8   GENDER         object 
dtypes: float64(1), int64(2), object(6)
memory usage: 614.8+ MB


In [6]:
def random_multi_choice(pool, max_items=2):
    k = random.randint(1, max_items)
    return ', '.join(random.sample(pool, k))
    
def clean_med_list_to_string(meds):
    remove_terms = {
        'mg', 'ml', 'units', 'sulfate', 'chloride', 'flush', 'g', 'mcg', 'hr','d10w','sw','mini','syringe',
        'meq', 'wcodeine', 'd5w', 'ns', 'cv1', 'cvl', 'sodium', 'potassium','bag','vial','send','100mcg','500mg'
        'senna', 'thiamine', 'tuberculin', 'pneumococcal', 'glucose', 'citrate', 'unitsml','d5','12ns','125gm'
    }
    
    tokens = []

    if isinstance(meds, str):
        # Remove brackets and punctuation, then split
        meds_cleaned = re.sub(r"[^\w\s]", "", meds)
        tokens = meds_cleaned.split()

    elif isinstance(meds, list):
        # Flatten if nested
        flat = []
        for item in meds:
            if isinstance(item, list):
                flat.extend(item)
            else:
                flat.append(item)
        # Convert to string tokens
        tokens = [str(t) for t in flat]

    else:
        return ''

    cleaned = []
    for token in tokens:
        token_lower = token.lower()
        
        # Remove if numeric or pattern like 500mg
        if (
            token_lower in remove_terms or
            re.fullmatch(r"\d+\.?\d*(mg|ml|mcg|g|meq|units)?", token_lower)
        ):
            continue
        
        cleaned.append(token.capitalize())
        
    return ' '.join(dict.fromkeys(cleaned))
        
# ------------------------------------------------
# STEP 2: Merge synthetic patient data back into full DataFrame
# ------------------------------------------------
df = df.merge(unique_subjects, on=['SUBJECT_ID', 'GENDER'], how='left')


effectiveness_levels = ['High', 'Moderate', 'Low']
adverse_reactions_pool = ['None', 'Nausea', 'Rash', 'Fatigue']
durations = ['5 days', '7 days', '10 days', '14 days']
dosages = ['5mg', '10mg', '20mg', '50mg']

# Apply synthetic logic
df['Chronic_Conditions'] = df['Chronic_Conditions'].str.lower()
df['Chronic_Conditions'] = df['DIAGNOSIS'].fillna('Unknown')
df['Recommended_Medication'] = df['DRUG'].apply(clean_med_list_to_string)
df['NDC'] = df['NDC'].astype(str).str.replace(r'\.0$', '', regex=True)

df['Symptoms'] = df['SHORT_TITLE'].fillna('Unknown')
df['Dosage'] = df['PROD_STRENGTH']
df['Duration'] = [random.choice(durations) for _ in range(num_rows)]
df['Treatment_Effectiveness'] = [random.choices(effectiveness_levels, weights=[0.6, 0.3, 0.1])[0] for _ in range(num_rows)]
df['Adverse_Reactions'] = [random.choices(adverse_reactions_pool, weights=[0.5, 0.2, 0.2, 0.1])[0] for _ in range(num_rows)]
df['Recovery_Time_Days'] = df['Treatment_Effectiveness'].map({'High': 5, 'Moderate': 10, 'Low': 15})

# print(df.info())
# Select and rename final columns
final_df = df[[
    'SUBJECT_ID', 'Age', 'GENDER', 'Weight_kg', 'Height_cm', 'BMI',
    'Chronic_Conditions', 'Drug_Allergies', 
    'ICD9_CODE', 'DIAGNOSIS', 'Symptoms','NDC',
    'Recommended_Medication', 'Dosage', 'Duration',
    'Treatment_Effectiveness', 'Adverse_Reactions', 'Recovery_Time_Days'
]].rename(columns={
    'SUBJECT_ID': 'PatientID',
    'GENDER': 'Gender',
    'ICD9_CODE': 'DIAGNOSIS',
    'DIAGNOSIS': 'Diagnosis_Description'
})
# final_df = final_df.dropna(subset=[
#     'Recommended_Medication', 'Symptoms', 'DIAGNOSIS'
# ])

In [8]:
final_df = df[[
    'SUBJECT_ID', 'Age', 'GENDER', 'Weight_kg', 'Height_cm', 'BMI',
    'Chronic_Conditions', 'Drug_Allergies', 
    'ICD9_CODE', 'Symptoms','NDC',
    'Recommended_Medication', 'Dosage', 'Duration',
    'Treatment_Effectiveness', 'Adverse_Reactions', 'Recovery_Time_Days'
]].rename(columns={
    'SUBJECT_ID': 'PatientID',
    'GENDER': 'Gender',
    'ICD9_CODE': 'Diagnosis'
    
})

In [10]:
final_df.drop_duplicates(inplace=True)

In [11]:
final_df

,PatientID,Age,Gender,Weight_kg,Height_cm,BMI,Chronic_Conditions,Drug_Allergies,Diagnosis,Symptoms,NDC,Recommended_Medication,Dosage,Duration,Treatment_Effectiveness,Adverse_Reactions,Recovery_Time_Days
0,2,23,M,43.6,156.0,17.9,NEWBORN,"Aspirin, Shellfish",V3001,Single lb in-hosp w cs,63323017302,Neoivgentamicin,10mg/mL-2mL,10 days,High,Nausea,5
1,2,23,M,43.6,156.0,17.9,NEWBORN,"Aspirin, Shellfish",V3001,Single lb in-hosp w cs,0,Neonatal,1 Syringe,5 days,Moderate,None,10
2,2,23,M,43.6,156.0,17.9,NEWBORN,"Aspirin, Shellfish",V3001,Single lb in-hosp w cs,63323038810,Ampicillin,500mg Vial,14 days,Low,Nausea,15
3,2,23,M,43.6,156.0,17.9,NEWBORN,"Aspirin, Shellfish",V3001,Single lb in-hosp w cs,0,,Send 500mg Vial,14 days,Moderate,Nausea,10
4,2,23,M,43.6,156.0,17.9,NEWBORN,"Aspirin, Shellfish",V053,Need prphyl vc vrl hepat,63323017302,Neoivgentamicin,10mg/mL-2mL,5 days,High,None,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8953461,10566,87,F,62.8,167.0,22.5,NEWBORN,"Penicillin, Shellfish",V0259,Bacteria dis carrier NEC,49281029810,Neoimdt Acellular Pertussis,0.5mL IM Dose,14 days,High,None,5
8953462,10566,87,F,62.8,167.0,22.5,NEWBORN,"Penicillin, Shellfish",V0259,Bacteria dis carrier NEC,5197067,Neoimpneumococcal 7valent,0.5mLVial,7 days,High,None,5
8953463,10566,87,F,62.8,167.0,22.5,NEWBORN,"Penicillin, Shellfish",V0259,Bacteria dis carrier NEC,182138167,Neopoferrous Elixir,50mL Bottle,7 days,High,None,5
8953464,10566,87,F,62.8,167.0,22.5,NEWBORN,"Penicillin, Shellfish",V0259,Bacteria dis carrier NEC,24208073501,Cyclopentolate,2 mL Dropper Bottle,5 days,High,Nausea,5


In [311]:
final_df = final_df.dropna(subset=[
    'Recommended_Medication', 'Symptoms', 'Chronic_Conditions'
])

In [313]:
final_df

,PatientID,Age,Gender,Weight_kg,Height_cm,BMI,Chronic_Conditions,Drug_Allergies,Diagnosis,Symptoms,NDC,Recommended_Medication,Dosage,Duration,Treatment_Effectiveness,Adverse_Reactions,Recovery_Time_Days
0,2,20,M,74.8,176.2,24.1,NEWBORN,Shellfish,V3001,Single lb in-hosp w cs,63323017302,Neoivgentamicin,10mg/mL-2mL,14 days,Moderate,None,10
1,2,20,M,74.8,176.2,24.1,NEWBORN,Shellfish,V3001,Single lb in-hosp w cs,0,Neonatal,1 Syringe,5 days,High,None,5
2,2,20,M,74.8,176.2,24.1,NEWBORN,Shellfish,V3001,Single lb in-hosp w cs,63323038810,Ampicillin,500mg Vial,5 days,High,None,5
3,2,20,M,74.8,176.2,24.1,NEWBORN,Shellfish,V3001,Single lb in-hosp w cs,0,,Send 500mg Vial,7 days,Low,None,15
4,2,20,M,74.8,176.2,24.1,NEWBORN,Shellfish,V053,Need prphyl vc vrl hepat,63323017302,Neoivgentamicin,10mg/mL-2mL,5 days,Moderate,Fatigue,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8953461,10566,19,F,62.9,175.0,20.5,NEWBORN,None,V0259,Bacteria dis carrier NEC,49281029810,Neoimdt Acellular Pertussis,0.5mL IM Dose,10 days,Moderate,Rash,10
8953462,10566,19,F,62.9,175.0,20.5,NEWBORN,None,V0259,Bacteria dis carrier NEC,5197067,Neoimpneumococcal 7valent,0.5mLVial,7 days,High,Nausea,5
8953464,10566,19,F,62.9,175.0,20.5,NEWBORN,None,V0259,Bacteria dis carrier NEC,24208073501,Cyclopentolate,2 mL Dropper Bottle,10 days,Low,Fatigue,15
8953465,10566,19,F,62.9,175.0,20.5,NEWBORN,None,V0259,Bacteria dis carrier NEC,65035902,Cyclopentolatephenylephrine,0.2%-1%;2ML,10 days,Moderate,Rash,10


In [14]:
final_df['Recommended_Medication_tuple'] = final_df['Recommended_Medication'].apply(tuple)
 

df_unique_rx = final_df.drop_duplicates(subset=[
    'PatientID', 'Diagnosis', 'Recommended_Medication_tuple'
])
# df_unique_rx
df_final = df_unique_rx.groupby('PatientID').agg({
    'Age': 'first',
    'Gender': 'first',
    'BMI': 'first',
    'Weight_kg':'first',
    'Height_cm':'first',
    'BMI':'first',
    'Chronic_Conditions':'first',
    'Symptoms':'first',
    'Diagnosis': 'first',
    'Recommended_Medication': 'first',
    'NDC':'first',
    'Dosage':'first',
    'Duration':'first',
    'Treatment_Effectiveness':'first', 
    'Adverse_Reactions':'first',
    'Recovery_Time_Days':'first'
    
}).reset_index()

df_final['Chronic_Conditions'] = df_final['Chronic_Conditions'].str.lower()
print(df_final.head())


   PatientID  Age Gender   BMI  Weight_kg  Height_cm  \
0          2   23      M  17.9       43.6      156.0   
1          4   37      F  25.4       68.0      163.5   
2          6   54      F  32.3       87.4      164.4   
3          8   77      M  26.6       82.7      176.3   
4          9   34      M  29.0       82.5      168.8   

                    Chronic_Conditions                  Symptoms Diagnosis  \
0                              NEWBORN    Single lb in-hosp w cs     V3001   
1  FEVER,DEHYDRATION,FAILURE TO THRIVE   Adv eff antiviral drugs     E9317   
2            CHRONIC RENAL FAILURE/SDA    Hyp kid NOS w cr kid V     40391   
3                              NEWBORN  Need prphyl vc vrl hepat      V053   
4                      HEMORRHAGIC CVA  Intracerebral hemorrhage       431   

         Recommended_Medication          NDC       Dosage Duration  \
0               Neoivgentamicin  63323017302  10mg/mL-2mL  10 days   
1  Guaifenesincodeine Phosphate     31867412    5ML UD

In [16]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7394 entries, 0 to 7393
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   PatientID                7394 non-null   int64  
 1   Age                      7394 non-null   int64  
 2   Gender                   7394 non-null   object 
 3   BMI                      7394 non-null   float64
 4   Weight_kg                7394 non-null   float64
 5   Height_cm                7394 non-null   float64
 6   Chronic_Conditions       7394 non-null   object 
 7   Symptoms                 7394 non-null   object 
 8   Diagnosis                7394 non-null   object 
 9   Recommended_Medication   7394 non-null   object 
 10  NDC                      7394 non-null   object 
 11  Dosage                   7394 non-null   object 
 12  Duration                 7394 non-null   object 
 13  Treatment_Effectiveness  7394 non-null   object 
 14  Adverse_Reactions       

In [18]:
df_final['Chronic_Conditions'] = df_final['Chronic_Conditions'].str.lower()
print(df_final.head())

   PatientID  Age Gender   BMI  Weight_kg  Height_cm  \
0          2   23      M  17.9       43.6      156.0   
1          4   37      F  25.4       68.0      163.5   
2          6   54      F  32.3       87.4      164.4   
3          8   77      M  26.6       82.7      176.3   
4          9   34      M  29.0       82.5      168.8   

                    Chronic_Conditions                  Symptoms Diagnosis  \
0                              newborn    Single lb in-hosp w cs     V3001   
1  fever,dehydration,failure to thrive   Adv eff antiviral drugs     E9317   
2            chronic renal failure/sda    Hyp kid NOS w cr kid V     40391   
3                              newborn  Need prphyl vc vrl hepat      V053   
4                      hemorrhagic cva  Intracerebral hemorrhage       431   

         Recommended_Medication          NDC       Dosage Duration  \
0               Neoivgentamicin  63323017302  10mg/mL-2mL  10 days   
1  Guaifenesincodeine Phosphate     31867412    5ML UD

In [20]:
df_final.to_csv('patient_icd_rx_grouped.csv', index=False)

In [300]:
def clean_med_list_to_string(meds):
    remove_terms = {
        'mg', 'ml', 'units', 'sulfate', 'chloride', 'flush', 'g', 'mcg', 'hr','d10w','sw','mini','syringe',
        'meq', 'wcodeine', 'd5w', 'ns', 'cv1', 'cvl', 'sodium', 'potassium','bag','vial','send','100mcg','500mg'
        'senna', 'thiamine', 'tuberculin', 'pneumococcal', 'glucose', 'citrate', 'unitsml','d5','12ns','125gm'
    }
    
    tokens = []

    if isinstance(meds, str):
        # Remove brackets and punctuation, then split
        meds_cleaned = re.sub(r"[^\w\s]", "", meds)
        tokens = meds_cleaned.split()

    elif isinstance(meds, list):
        # Flatten if nested
        flat = []
        for item in meds:
            if isinstance(item, list):
                flat.extend(item)
            else:
                flat.append(item)
        # Convert to string tokens
        tokens = [str(t) for t in flat]

    else:
        return ''

    cleaned = []
    for token in tokens:
        token_lower = token.lower()
        
        # Remove if numeric or pattern like 500mg
        if (
            token_lower in remove_terms or
            re.fullmatch(r"\d+\.?\d*(mg|ml|mcg|g|meq|units)?", token_lower)
        ):
            continue
        
        cleaned.append(token.capitalize())
        
    return ' '.join(dict.fromkeys(cleaned))